<a href="https://colab.research.google.com/github/vjmiyagi/DS-Unit-1-Sprint-2-Statistics/blob/master/module2/VJM_LS_DSPT6_122_Sampling_Confidence_Intervals_and_Hypothesis_Testing_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment - Build a confidence interval

A confidence interval refers to a neighborhood around some point estimate, the size of which is determined by the desired p-value. For instance, we might say that 52% of Americans prefer tacos to burritos, with a 95% confidence interval of +/- 5%.

52% (0.52) is the point estimate, and +/- 5% (the interval $[0.47, 0.57]$) is the confidence interval. "95% confidence" means a p-value $\leq 1 - 0.95 = 0.05$.

In this case, the confidence interval includes $0.5$ - which is the natural null hypothesis (that half of Americans prefer tacos and half burritos, thus there is no clear favorite). So in this case, we could use the confidence interval to report that we've failed to reject the null hypothesis.

But providing the full analysis with a confidence interval, including a graphical representation of it, can be a helpful and powerful way to tell your story. Done well, it is also more intuitive to a layperson than simply saying "fail to reject the null hypothesis" - it shows that in fact the data does *not* give a single clear result (the point estimate) but a whole range of possibilities.

How is a confidence interval built, and how should it be interpreted? It does *not* mean that 95% of the data lies in that interval - instead, the frequentist interpretation is "if we were to repeat this experiment 100 times, we would expect the average result to lie in this interval ~95 times."

For a 95% confidence interval and a normal(-ish) distribution, you can simply remember that +/-2 standard deviations contains 95% of the probability mass, and so the 95% confidence interval based on a given sample is centered at the mean (point estimate) and has a range of +/- 2 (or technically 1.96) standard deviations.

Different distributions/assumptions (90% confidence, 99% confidence) will require different math, but the overall process and interpretation (with a frequentist approach) will be the same.

Your assignment - using the data from the prior module ([congressional voting records](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records)):


### Confidence Intervals:
1. Generate and numerically represent a confidence interval
2. Graphically (with a plot) represent the confidence interval
3. Interpret the confidence interval - what does it tell you about the data and its distribution?

### Chi-squared tests:
4. Take a dataset that we have used in the past in class that has **categorical** variables. Pick two of those categorical variables and run a chi-squared tests on that data
  - By hand using Numpy
  - In a single line using Scipy


In [0]:
#@title This cell contains imports
# imports
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from matplotlib import rc

In [0]:
#@title This cell gets data with bang command
# Get the data

# wget
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data --no-check-certificate

In [0]:
#@title This cell makes the dataframe and prints head
# Make DataFrame
cols = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']
house_df = pd.read_csv('house-votes-84.data',header=None,
                    names=cols)
house_df.head()

In [0]:
#@title This cell cleans data using NaN for ?
house_df = house_df.replace({'y':1, 'n':0, '?':np.NaN})
house_df.head()

In [0]:
#@title This cell makes two dataframes for the two parties
dem_df = house_df[house_df['party']=='democrat']
print('dem',dem_df.shape)
rep_df = house_df[house_df['party']=='republican']
print('rep',rep_df.shape)

In [0]:
#@title This cell defines function to compute confidence interval 

def conf_int(df, confidence=0.95):
    df = np.array(df)
    mean = np.nanmean(df)
    n = len(df)
    stderr = np.nanstd(df, ddof=1)/np.sqrt(n)
    moe = stderr * stats.t.ppf((1 + confidence)/ 2.0, n-1)
    if mean >=0.475  and mean <= 0.525:
        str = 'fail to reject'
    else:
        str = 'reject'
    return(round(mean,4), round(mean-moe,4), round(mean+moe,4),str)

In [0]:
#@title Use this cell to select a bill to analyze.
bill = "handicapped-infants" #@param ["handicapped-infants", "water-project", "budget", "physician-fee-freeze", "el-salvador-aid", "religious-groups", "anti-satellite-ban", "aid-to-contras", "mx-missile", "immigration", "synfuels", "education", "right-to-sue", "crime", "duty-free", "south-africa"]

bill.title()

In [0]:
#@title This cell calls the  function to compute confidence interval 

CI_dem = conf_int(dem_df[bill])
CI_rep = conf_int(rep_df[bill])

print('Regarding the house vote for',bill.title())
print('with', house_df[bill].sum(), 'votes,',
      'looking at just Democrats:')
print('with a mean of',CI_dem[0],'and using a 95% confidence level')
print('we',CI_dem[3],'the null hypothesis')
print()
print('When we look at the Republicans', 'the mean is', CI_rep[0])
print('so we',CI_rep[3],'the null hypothesis.')

In [0]:
#@title This cell draws a graph of votes on the selected bill
fig, ax = plt.subplots()



sns.kdeplot(house_df[bill],
            bw='.1', shade=True,
            label='house', color = 'g', ax=ax);

sns.kdeplot(dem_df[bill],
            bw='.1', shade=True,
            label='dem', color = 'b', ax=ax);
sns.kdeplot(rep_df[bill],
            bw='.1', shade=True,
            label='rep', color = 'r', ax=ax);

plt.xticks([0,1], ['no', 'yes'] );
plt.axvline(x=CI_dem[1], color='b')
plt.axvline(x=CI_dem[2], color='b')
plt.axvline(x=CI_dem[0],color='c')
plt.axvline(x=CI_rep[1], color='r')
plt.axvline(x=CI_rep[2], color='r')
plt.axvline(x=CI_rep[0], label='rep mean',color='m')
graphtitle=(bill.title()+' House votes '+str(int(house_df[bill].sum())))
graphsubtitle=('Republicans : ' + str(int(rep_df[bill].sum()))+ 
               '                          ' +
               'Democrats : ' + str(int(dem_df[bill].sum())))
plt.title(graphsubtitle)
plt.suptitle(graphtitle)

plt.legend

plt.show()

In [0]:
#@title This Cell computes Chi square test using Numpy


odf = pd.crosstab(house_df['party'], house_df[bill], margins=True)

# Set Observed Values
Oa=odf.iloc[0,0]
Ob=odf.iloc[0,1]
Oc=odf.iloc[1,0]
Od=odf.iloc[1,1]
# Set Expected Values
Ea = (odf.iloc[0,2] * odf.iloc[2,0])/odf.iloc[2,2]
Eb = (odf.iloc[0,2] * odf.iloc[2,1])/odf.iloc[2,2]
Ec = (odf.iloc[1,2] * odf.iloc[2,0])/odf.iloc[2,2]
Ed = (odf.iloc[1,2] * odf.iloc[2,1])/odf.iloc[2,2]

ChiSq = ((Oa-Ea)**2/Ea)+((Ob-Eb)**2/Eb)+((Oc-Ec)**2/Ec)+((Od-Ed)**2/Ed)
print('Chi square =', ChiSq)


In [0]:
#@title This Cell computes Chi square test In a single line using Scipy

stats.chi2_contingency(odf)

## Stretch goals:

1. Write a summary of your findings, mixing prose and math/code/results. *Note* - yes, this is by definition a political topic. It is challenging but important to keep your writing voice *neutral* and stick to the facts of the data. Data science often involves considering controversial issues, so it's important to be sensitive about them (especially if you want to publish).
2. Apply the techniques you learned today to your project data or other data of your choice, and write/discuss your findings here.
3. Refactor your code so it is elegant, readable, and can be easily run for all issues.

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)